In [ ]:
'''
A Multilayer Perceptron implementation example using TensorFlow library.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
'''

In [1]:
# Import MINST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

import tensorflow as tf

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


학습을 위한 파라미터들을 설정한다.

In [ ]:
# Parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100
display_step = 1

네트워크 구성을 위한 파라미터를 설정한다.
1번째, 2번째 히든 노드는 256개의 피쳐를 사용하고, 입력차원은 784차원, 출력차원은 10차원으로 설정한다.

In [ ]:
# Network Parameters
n_hidden_1 = 256 # 1st layer number of features
n_hidden_2 = 256 # 2nd layer number of features
n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)

x와 y에 대한 placeholder를 설정한다.

In [2]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

Multi Layer Perception을 weights와 biases 값들이 들어오면 만들어주는 함수를 만들어준다.
여기선 범용성있게 설계된 건 아니고, 2층의 dense layer와 1층의 output layer로 구성하였고,
각 layer 를 통과한 이후 activation 으로 relu (tf.nn.relu)를 사용하였다.

In [3]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

각 변수들은 tf.random_normal을 통해 normal 분포를 따르는 임의의 값으로 초기화되며, 해당 함수의 인자로써 변수의 모양이 정의된다.
아래는 각 레이어별로 학습될 변수들 h, b, out 등을 설정한다.

설정된 값을 바탕으로

1. multilayer_perceptron을 구성하고
2. cost로 multiclass 분류문제에 사용하는 cross entropy를 쓰고 (tf.nn.softmax_cross_entropy_with_logits)
3. optimizer로 고정된 learning_rate로 cost를 최소화하도록 설정된 AdamOptimizer를 사용하도록 한다.

In [5]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

batch_size 단위로 파일을 읽으면서 모델을 학습시켜 나갑니다.

1. total_batch를 전체 example수로부터 구합니다.
2. batch_x, batch_y에 각 배치별 입력을 할당합니다.
3. sess.run을 통해 optimizer를 돌리고, 그 때 계산된 cost를 반환받습니다.
4. 전체에 대한 평균 cost를 구하기 위해 avg_cost를 계산합니다.
5. 특정 epoch 주기마다 avg_cost를 출력하여줍니다.
6. accuracy를 tf의 연산자들의 조합으로 구하고 이를 출력합니다.
    * 이 때 accuracy.eval은 해당 연산에 대해 sess.run을 하는 것과 같은 역할을 하고, 인자로 들어간 것은 feed_dict와 같은 역할을 합니다.

In [6]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={x: batch_x,
                                                          y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"

    # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels})

Epoch: 0001 cost= 173.056566575
Epoch: 0002 cost= 44.054413928
Epoch: 0003 cost= 27.455470655
Epoch: 0004 cost= 19.008652363
Epoch: 0005 cost= 13.654873594
Epoch: 0006 cost= 10.059267435
Epoch: 0007 cost= 7.436018432
Epoch: 0008 cost= 5.587794416
Epoch: 0009 cost= 4.209882509
Epoch: 0010 cost= 3.203879515
Epoch: 0011 cost= 2.319920681
Epoch: 0012 cost= 1.676204545
Epoch: 0013 cost= 1.248805338
Epoch: 0014 cost= 1.052676844
Epoch: 0015 cost= 0.890117338
Optimization Finished!
Accuracy: 0.9459
